# Hybrid Recommender System

prototype

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import tensorflow as tf
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, BatchNormalization, Dropout
from keras.models import Model
from keras.optimizers import Adam
from sklearn.metrics.pairwise import cosine_similarity

2023-04-05 06:57:26.749269: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 06:57:26.881311: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def load_data():
    table_df = pd.read_csv('../data/ulsan_rest_table_ver3.csv')
    print(table_df.isnull().sum())
    table_df.drop(columns='comment', inplace=True)
    table_df.dropna(axis=0, how='any', inplace=True)
    data_df = table_df[['u_id', 'p_id', 'score']]
    return data_df

In [3]:
def culc_sim():
    item_Table = pd.read_csv('../data/kmeans_item_Table.csv')
    item_Matrix = item_Table.groupby('p_id').mean()
    item_Matrix = item_Matrix.to_numpy()
    sim = cosine_similarity(item_Matrix, item_Matrix)
    return sim

In [4]:
def get_unvisted_item(data_df, u_id):
    visited_items = set(data_df[data_df.u_id == u_id]['p_id'].tolist())
    unvisited_items = [item for item in data_df.p_id.unique() if item not in visited_items]
    return unvisited_items

In [5]:
def get_items(data_df, u_id):
    visited_items = set(data_df[data_df.u_id == u_id]['p_id'].tolist())
    unvisited_items = [item for item in data_df.p_id.unique() if item not in visited_items]
    return unvisited_items, visited_items

In [6]:
def recom_cf(model, u_id, data_df, top_n):

    p_id_list = get_unvisted_item(data_df, u_id)
    pred_df = pd.DataFrame(columns=['u_id', 'p_id', 'score'])
            
    for p_id in p_id_list:
        pred = model.predict([np.array([u_id]), np.array([p_id])])
        new_data = pd.DataFrame({'u_id': [u_id], 'p_id': [p_id], 'score': [float(pred)]})
        pred_df = pd.concat([pred_df, new_data], axis=0, names=['u_id', 'p_id', 'score'], ignore_index=True)
    
    recom_df = pred_df.sort_values(by=['score'], ascending=False)[:top_n]

    return recom_df

In [7]:
def recom_cbf(p_id, sim):
    sim_scores = list(enumerate(sim[p_id]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]

    return sim_scores

In [8]:
def recom_hybrid(model, u_id, data_df, top_n, sim):
    unvisited_items, visited_items = get_items(data_df, u_id)
    visited_items = set(data_df[data_df.u_id == u_id]['p_id'].tolist())
    pred_df = pd.DataFrame(columns=['u_id', 'p_id', 'score'])
    
    recom_cbf_pids = []
    for cbf_p_id in visited_items:
        for item in recom_cbf(cbf_p_id, sim): #컨텐츠기반필터링
            recom_cbf_pids.append(item[0])
    recom_cbf_pids = set(recom_cbf_pids)
    
    for p_id in unvisited_items:
        pred = model.predict([np.array([u_id]), np.array([p_id])]) #협업필터링
        if p_id in recom_cbf_pids: #컨텐츠기반필터링 추천 목록에 포함된다면..
            pred = pred + pred*0.05     # 가중치 연산
        new_data = pd.DataFrame({'u_id': [u_id], 'p_id': [p_id], 'score': [float(pred)]})
        pred_df = pd.concat([pred_df, new_data], axis=0, names=['u_id', 'p_id', 'rating'], ignore_index=True)
    
    recom_df = pred_df.sort_values(by=['score'], ascending=False)[:top_n]    
    return recom_df

In [9]:
loaded_model = tf.keras.models.load_model("../Filtering/model/MLP.h5")
u_id = 10556
data_df = load_data()
sim = culc_sim()
top_n = recom_hybrid(loaded_model, u_id, data_df, 10, sim)
top_n

2023-04-05 06:57:29.331314: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 06:57:30.073442: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46695 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:af:00.0, compute capability: 8.6


p_id            0
place_id        0
u_id            0
user_id         0
score           0
comment     14275
dtype: int64
1/1 [==============================] - 0s 28ms/step


2023-04-05 06:57:32.653863: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 0s 29ms/step


,u_id,p_id,score
6,10556,6,5.167598
398,10556,438,5.029439
116,10556,127,4.988582
172,10556,189,4.982726
416,10556,460,4.924480
41,10556,48,4.916735
7,10556,8,4.902934
431,10556,478,4.892490
234,10556,259,4.848725
353,10556,390,4.839103
